
# LLM Interface Concept
In this notebook, we will explore the concept of creating a **unified interface** for interacting with various Large Language Models (LLMs).
This approach makes it easier to switch between different LLMs (e.g., OpenAI, Anthropic's Claude, Cohere) using a common interface.

An **interface** in programming defines a **set of methods** that a class must implement, ensuring that different implementations can be accessed in the same way.
We will also demonstrate how LangChain handles this concept.

### Key Points:
- **Interface**: A contract that defines methods to interact with models in a unified way.
- **Benefits**: Allows switching between LLMs without changing application code.


In [ ]:
from abc import ABC, abstractmethod

# Define an interface (abstract class) for LLMs
class LLMInterface(ABC):

    @abstractmethod
    def generate(self, prompt: str) -> str:
        # Generate a response from the LLM.
        pass


In [ ]:
import openai

class OpenAILLM(LLMInterface):
    def __init__(self, api_key):
        openai.api_key = api_key

    def generate(self, prompt: str) -> str:
        response = openai.Completion.create(
            engine="text-davinci-003",
            prompt=prompt,
            max_tokens=150,
            temperature=0.7
        )
        return response.choices[0].text.strip()


In [ ]:
import requests

class ClaudeLLM(LLMInterface):
    def __init__(self, api_key):
        self.api_key = api_key
        self.api_url = "https://api.anthropic.com/v1/complete"

    def generate(self, prompt: str) -> str:
        headers = {
            'Authorization': f'Bearer {self.api_key}',
            'Content-Type': 'application/json',
        }
        data = {
            'prompt': prompt,
            'model': 'claude-v1',
            'max_tokens_to_sample': 150,
            'temperature': 0.7,
        }
        response = requests.post(self.api_url, headers=headers, json=data)
        return response.json().get('completion', "No response")


In [ ]:
import cohere

class CohereLLM(LLMInterface):
    def __init__(self, api_key):
        self.client = cohere.Client(api_key)

    def generate(self, prompt: str) -> str:
        response = self.client.generate(
            model="command-xlarge-nightly",
            prompt=prompt,
            max_tokens=150,
            temperature=0.7
        )
        return response.generations[0].text.strip()


In [ ]:
# Choose an LLM dynamically
def get_llm(provider, api_key):
    if provider == "openai":
        return OpenAILLM(api_key)
    elif provider == "claude":
        return ClaudeLLM(api_key)
    elif provider == "cohere":
        return CohereLLM(api_key)
    else:
        raise ValueError("Invalid provider")

# Example usage
llm = get_llm("openai", "YOUR_OPENAI_API_KEY")
response = llm.generate("What is artificial intelligence?")
print(response)


In [ ]:
# Example with LangChain
from langchain.llms import OpenAI

# Initialize OpenAI model with API key
llm = OpenAI(api_key="YOUR_OPENAI_API_KEY")

# Generate response
response = llm("What is artificial intelligence?")
print(response)
